In [1]:
#Cuda compilation tools, release 11.0, V11.0.221
!pip install --upgrade mxnet-cu100

     |████████████████████████████████| 352.6 MB 13 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
!pip install "onnx<=1.9.0"

     |████████████████████████████████| 12.2 MB 248 kB/s 


In [3]:
import mxnet as mx
import onnx
print(onnx.__version__)
print(mx.__version__)

1.9.0
1.8.0


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [12]:
!unzip /mydrive/dataset.zip -d /mydrive/

Archive:  /mydrive/dataset.zip
   creating: /mydrive/dataset/
   creating: /mydrive/dataset/train/
   creating: /mydrive/dataset/train/arduino/
  inflating: /mydrive/dataset/train/arduino/1.jpg  
  inflating: /mydrive/dataset/train/arduino/10.jpg  
  inflating: /mydrive/dataset/train/arduino/100.jpg  
  inflating: /mydrive/dataset/train/arduino/101.jpg  
  inflating: /mydrive/dataset/train/arduino/102.jpg  
  inflating: /mydrive/dataset/train/arduino/103.jpg  
  inflating: /mydrive/dataset/train/arduino/104.jpg  
  inflating: /mydrive/dataset/train/arduino/105.jpg  
  inflating: /mydrive/dataset/train/arduino/106.jpg  
  inflating: /mydrive/dataset/train/arduino/107.jpg  
  inflating: /mydrive/dataset/train/arduino/108.jpg  
  inflating: /mydrive/dataset/train/arduino/109.jpg  
  inflating: /mydrive/dataset/train/arduino/11.jpg  
  inflating: /mydrive/dataset/train/arduino/110.jpg  
  inflating: /mydrive/dataset/train/arduino/111.jpg  
  inflating: /mydrive/dataset/train/arduino/112.jp

In [16]:
import math
import os
import time
from mxnet import autograd
from mxnet import gluon, init
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from mxnet.gluon.model_zoo.vision import *
import mxnet as mx
from mxnet.contrib import onnx as onnx_mxnet
import numpy as np

#edit with your dataset 
output_dir = "/mydrive/output_training"
dataset_dir = "/mydrive/dataset"
pretrain_name = "vgg11_bn"
width = 224
height = 224
batch_per_device = 32
num_workers = 4
num_class = 4
lr = 0.001
log_print = 20
epochs = 10

os.makedirs(output_dir)
print('using gpu for training.')
num_gpus = mx.context.num_gpus()
print('number of gpu : ',num_gpus)
ctx = [mx.gpu(i) for i in range(num_gpus)]
batch_size = batch_per_device * max(num_gpus, 1)
print('Batch size for training : ',batch_size)

training_transformer = transforms.Compose([
    transforms.RandomResizedCrop(width),
    transforms.RandomFlipLeftRight(),
    transforms.RandomColorJitter(brightness=0.4, contrast=0.4,saturation=0.4),
    transforms.RandomLighting(0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

validation_transformer = transforms.Compose([
    transforms.Resize(width),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

train_path = os.path.join(dataset_dir, 'train')
val_path = os.path.join(dataset_dir, 'valid')
test_path = os.path.join(dataset_dir, 'test')
list_label = sorted(os.listdir(train_path))
if not os.path.exists(output_dir+'/labels.txt'):
    with open(output_dir+'/labels.txt', 'w') as f:
        for single_label in list_label:
            f.write(single_label)
            f.write("\n")
    f.close()

train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(train_path).transform_first(training_transformer),
    batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(val_path).transform_first(validation_transformer),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(test_path).transform_first(validation_transformer),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)

if pretrain_name == 'resnet18_v1':
    net = resnet18_v1(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet34_v1':
    net = resnet34_v1(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet50_v1':
    net = resnet50_v1(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet101_v1':
    net = resnet101_v1(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet152_v1':
    net = resnet152_v1(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet18_v2':
    net = resnet18_v2(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet34_v2':
    net = resnet34_v2(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet50_v2':
    net = resnet50_v2(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet101_v2':
    net = resnet101_v2(pretrained=True,ctx=ctx)
elif pretrain_name == 'resnet152_v2':
    net = resnet152_v2(pretrained=True,ctx=ctx)
elif pretrain_name == 'alexnet':
    net = alexnet(pretrained=True,ctx=ctx)
elif pretrain_name == 'densenet121':
    net = densenet121(pretrained=True,ctx=ctx)
elif pretrain_name == 'densenet161':
    net = densenet161(pretrained=True,ctx=ctx)
elif pretrain_name == 'densenet169':
    net = densenet169(pretrained=True,ctx=ctx)
elif pretrain_name == 'densenet201':
    net = densenet201(pretrained=True,ctx=ctx)
elif pretrain_name == 'inception_v3':
    net = inception_v3(pretrained=True,ctx=ctx)
    if width != 299 or height != 299:
        print("Fine tuning with inception_v3 must set width=height=299.")
elif pretrain_name == 'vgg11':
    net = vgg11(pretrained=True,ctx=ctx)
elif pretrain_name == 'vgg13':
    net = vgg13(pretrained=True,ctx=ctx)
elif pretrain_name == 'vgg16':
    net = vgg16(pretrained=True,ctx=ctx)
elif pretrain_name == 'vgg19':
    net = vgg19(pretrained=True,ctx=ctx)
elif pretrain_name == 'vgg11_bn':
    net = vgg11_bn(pretrained=True,ctx=ctx)
elif pretrain_name == 'vgg13_bn':
    net = vgg13_bn(pretrained=True,ctx=ctx)
elif pretrain_name == 'vgg16_bn':
    net = vgg16_bn(pretrained=True,ctx=ctx)
elif pretrain_name == 'vgg19_bn':
    net = vgg19_bn(pretrained=True,ctx=ctx)
elif pretrain_name == 'squeezenet1_0':
    net = squeezenet1_0(pretrained=True,ctx=ctx)
elif pretrain_name == 'squeezenet1_1':
    net = squeezenet1_1(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet0_25':
    net = mobilenet0_25(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet0_5':
    net = mobilenet0_5(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet0_75':
    net = mobilenet0_75(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet1_0':
    net = mobilenet1_0(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet_v2_0_25':
    net = mobilenet_v2_0_25(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet_v2_0_5':
    net = mobilenet_v2_0_5(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet_v2_0_75':
    net = mobilenet_v2_0_75(pretrained=True,ctx=ctx)
elif pretrain_name == 'mobilenet_v2_1_0':
    net = mobilenet_v2_1_0(pretrained=True,ctx=ctx)
else:
    print('Unknow model from pretrain ImageNet mxnet.')
with net.name_scope():
    net.output = nn.Dense(num_class)
net.output.initialize(init.Xavier(), ctx=ctx)
net.hybridize()
print(net)
print('model fine tune : ',pretrain_name)

num_batch = len(train_data)
iterations_per_epoch = math.ceil(num_batch)
lr_steps = [epoch * iterations_per_epoch for epoch in [5,10,15]]
schedule = mx.lr_scheduler.MultiFactorScheduler(step=lr_steps, factor=0.75, base_lr=lr)

sgd_optimizer = mx.optimizer.SGD(learning_rate=lr, lr_scheduler=schedule, momentum=0.9, wd=0.0001)
metric = mx.metric.Accuracy()
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), optimizer=sgd_optimizer)

def test(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for (data, label) in val_data:
        data = gluon.utils.split_and_load(data, ctx_list=ctx, even_split=False)
        label = gluon.utils.split_and_load(label, ctx_list=ctx, even_split=False)
        outputs = [net(x) for x in data]
        metric.update(label, outputs)
    return metric.get()

for epoch in range(epochs):
    tic = time.time()
    train_loss = 0
    train_loss_log = 0
    metric.reset()
    for i, (data, label) in enumerate(train_data):
        data = gluon.utils.split_and_load(data, ctx_list=ctx, even_split=False)
        label = gluon.utils.split_and_load(label, ctx_list=ctx, even_split=False)
        with autograd.record():
            outputs = [net(x) for x in data]
            loss = [softmax_cross_entropy(yhat, y) for yhat, y in zip(outputs, label)]
        for l in loss:
            l.backward()
        trainer.step(batch_size)
        train_loss += sum([l.mean().asscalar() for l in loss])/len(loss)
        metric.update(label, outputs)
        if i % log_print == 0:
            if i > 0:
                train_loss_log = train_loss
                current_loss = train_loss_log/i
                print("Epoch: %d; Batch %d; Loss %f" % (epoch, i, current_loss))
            else:
                print("Epoch: %d; Batch %d; Loss %f" % (epoch, i, train_loss))
    _, train_acc = metric.get()
    train_loss /= num_batch
    _, val_acc = test(net, val_data, ctx)
    print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc: %.3f | learning-rate: %.3E | time: %.1f' %(epoch, train_acc, train_loss, val_acc, trainer.learning_rate, time.time() - tic))
_, test_acc = test(net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))
net.export(output_dir+"/"+pretrain_name+"_custom",epoch=epochs)
sym = output_dir+'/'+pretrain_name+'_custom-symbol.json'
onnx_file = output_dir+'/'+pretrain_name+'_custom.onnx'
if epochs <=9:
    params = output_dir+'/'+pretrain_name+'_custom-000'+str(epochs)+'.params'
elif 9 < epochs <= 99:
    params = output_dir+'/'+pretrain_name+'_custom-00'+str(epochs)+'.params'
else:
    params = output_dir+'/'+pretrain_name+'_custom-0'+str(epochs)+'.params'
print(sym)
print(params)
print(onnx_file)
converted_model_path = onnx_mxnet.export_model(sym, params, [(1,3,width,height)], np.float32, onnx_file)


using gpu for training.
number of gpu :  1
Batch size for training :  32
VGG(
  (features): HybridSequential(
    (0): Conv2D(3 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (2): Activation(relu)
    (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
    (4): Conv2D(64 -> 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=128)
    (6): Activation(relu)
    (7): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
    (8): Conv2D(128 -> 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=256)
    (10): Activation(r

In [17]:
def download(path):
  from google.colab import files
  files.download(path)  

In [18]:
download('/mydrive/output_training/vgg11_bn_custom.onnx')
download('/mydrive/output_training/labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>